## 算分公式開發過程

### submit example

```
ETFid,Mon_ud,Mon_cprice,Tue_ud,Tue_cprice,Wed_ud,Wed_cprice,Thu_ud,Thu_cprice,Fri_ud,Fri_cprice
0050,1,87.15,1,88.05,-1,88.00,-1,87.25,1,87.45
0051,1,32.49,1,33,1,32.72,1,32.75,-1,32.44
0052,-1,57.75,1,58.5,1,59.8,-1,59,-1,57.9
0053,1,37.38,1,37.92,-1,37.44,-1,37.2,1,37.44
0054,1,24.63,1,24.94,-1,24.05,1,24.63,1,24.7
0055,-1,17.1,0,17.1,-1,17,0,17,1,17.02
0056,-1,26,1,26.25,-1,26.17,1,26.32,1,26.45
0057,1,52.4,1,52.9,-1,52.6,1,52.9,-1,52.05
0058,1,46.56,1,47.02,1,47.27,0,47.27,1,47.37
0059,-1,41.9,1,42.07,-1,41.44,0,41.44,1,41.9
006201,-1,14.98,1,15.27,-1,15.00,-1,14.97,-1,14.90
006203,1,40.24,1,40.84,-1,40.27,1,40.42,-1,40.40
006204,1,55.35,1,55.70,-1,55.35,1,55.40,1,55.45
006208,1,49.75,1,49.8,1,49.85,1,49.11,-1,49.35
00690,1,22.48,1,22.44,-1,22.24,1,22.28,-1,22.42
00692,1,22.14,1,22.23,-1,22.04,1,22.12,1,22.19
00701,1,21.46,1,21.5,-1,21.38,1,21.41,0,21.41
00713,-1,31.22,-1,31.1,-1,31.07,1,31.14,-1,31.12
```

In [1]:
import csv
import pandas as pd
import numpy as np
from collections import OrderedDict

### etf ID list

In [2]:
etfid_list = ['0051', '0050', '0052', '0053', '0054', '0055', '0056', '0057', 
              '0058', '0059', '006201', '006203', '006204', '006208', '00690', 
              '00692', '00701', '00713']

In [24]:
week_colunm_list = ['Mon_ud', 'Mon_cprice', 'Tue_ud', 'Tue_cprice', 'Wed_ud',
 'Wed_cprice', 'Thu_ud', 'Thu_cprice', 'Fri_ud', 'Fri_cprice']

In [39]:
ETF_NUM = len(etfid_list)
SUBMIT_COLNUM = len(week_colunm_list) + 1

### 計算一週單一個股的預測分數 (暫時不用)

In [5]:
### assume this are 0050 real values in a week ###
actual_0050 = OrderedDict({'20180122': (1, 87.15), '20180123': (1, 88.05), '20180124': (-1, 88.00), 
                '20180125': (-1, 87.25), '20180126': (1, 87.45)})

### assume this are 0050 predict values in a week ###
predict_0050 = OrderedDict({'20180122': (1, 87.45), '20180123': (1, 88.05), '20180124': (1, 88.3), 
                '20180125': (-1, 87.25), '20180126': (1, 88.7)})

In [20]:
def evaluate_temp(predict, actual):
    """ description 
      - 假設一週必定是 5 天，如果當週有放假日，可先在當天塞假資料 (ex. 都是 1 之類的)
      - score 的滿分是 1，但可能負無限大。如程式日期或天數發生錯誤，回傳 1000 表示 error
    """
    wrighting = [0.1, 0.15, 0.2, 0.25, 0.3]
    score = 0
    if predict.keys() == actual.keys() and len(predict) == 5:
        for i, j in enumerate(actual):
            day_score = 0.0
            if predict[j][0] == actual[j][0]:
                day_score += 0.5
            
            day_score += (((actual[j][1] - abs(predict[j][1] - actual[j][1]))/actual[j][1])*0.5) 
            score += day_score * wrighting[i]
    else:
        score = 1000
        
    return score
    
evaluate_temp(predict_0050, actual_0050)

0.8973428915367424

### 讀取官網範例 submit 格式

In [45]:
answer = pd.read_csv('../data/submit_example.csv', dtype={'ETFid' : np.str})

In [46]:
list(answer.columns)

['ETFid',
 'Mon_ud',
 'Mon_cprice',
 'Tue_ud',
 'Tue_cprice',
 'Wed_ud',
 'Wed_cprice',
 'Thu_ud',
 'Thu_cprice',
 'Fri_ud',
 'Fri_cprice']

In [91]:
answer

,ETFid,Mon_ud,Mon_cprice,Tue_ud,Tue_cprice,Wed_ud,Wed_cprice,Thu_ud,Thu_cprice,Fri_ud,Fri_cprice
0,0050,1,87.15,1,88.05,-1,88.00,-1,87.25,1,87.45
1,0051,1,32.49,1,33.00,1,32.72,1,32.75,-1,32.44
2,0052,-1,57.75,1,58.50,1,59.80,-1,59.00,-1,57.90
3,0053,1,37.38,1,37.92,-1,37.44,-1,37.20,1,37.44
4,0054,1,24.63,1,24.94,-1,24.05,1,24.63,1,24.70
5,0055,-1,17.10,0,17.10,-1,17.00,0,17.00,1,17.02
6,0056,-1,26.00,1,26.75,-1,26.17,1,26.32,1,26.45
7,0057,1,52.40,1,52.90,-1,52.10,1,52.90,-1,52.05
8,0058,1,46.56,1,47.02,1,47.27,0,47.27,1,47.37
9,0059,-1,41.90,1,42.07,-1,41.44,0,41.44,1,41.90


In [213]:
test_50_price = pd.Series([87.15, 88.05, 88.00, 87.25, 87.45])
target_50_price = pd.Series([87.15, 87.05, 86.00, 87.25, 87.45])

test_50_updown = pd.Series([1, 1, -1, -1, 1])
target_50_updown = pd.Series([1, 1, -1, 1, 1])

wieht = [0.1, 0.15, 0.2, 0.25, 0.3]

In [224]:
def evaluate_price(predict: pd.Series, target: pd.Series,
                   weight: list = [0.1, 0.15, 0.2, 0.25, 0.3], etf_code: str = "") -> float:
    result = ((target - abs(predict - target)) / target) * 0.5
    result.name = etf_code
    return result

price = evaluate_price(test_50_price, target_50_price, wieht, '0050')

In [215]:
def evaluate_updown(predict: pd.Series, target: pd.Series, etf_code: str) -> float:
    result = (predict == target).apply(lambda x:  0.5 if x is True else 0)
    result.name = etf_code
    return result
updown = evaluate_updown(test_50_updown, target_50_updown, '0050')

In [226]:
def evaluate_score(predict_cprice: pd.Series, 
                   target_cprice: pd.Series,
                   predict_updown: pd.Series, 
                   target_updown: pd.Series,
                   weight: list = [0.1, 0.15, 0.2, 0.25, 0.3],
                   etf_code: str = "") -> float:
    price = evaluate_price(predict_cprice, target_cprice, wieht, etf_code)
    updown = evaluate_updown(predict_updown, target_updown, etf_code)
    return sum(price) + sum(updown)
    


evaluate_score(test_50_price, target_50_price, wieht, test_50_updown, target_50_updown, '0050')

2.4826282676355476

In [112]:
price_list = []
updown_list = []

for row_index in range(len(answer)):
    this_series = answer.iloc[row_index]
    etf_code = this_series.pop('ETFid')
    this_price = pd.Series([this_series['Mon_cprice'], this_series['Tue_cprice'],this_series['Wed_cprice'],
                           this_series['Thu_cprice'],this_series['Fri_cprice']])
    this_price.name = etf_code
    #this_price = this_price.rename({0: 'Mon_cprice', 1: 'Tue_cprice', 2: 'Wed_cprice', 3: 'Thu_cprice', 4: 'Fri_cprice'})
    this_up_down = pd.Series([this_series['Mon_ud'], this_series['Tue_ud'],this_series['Wed_ud'],
                           this_series['Thu_ud'],this_series['Fri_ud']])
    this_up_down.name = etf_code
    #this_up_down = this_up_down.rename({0: 'Mon_ud', 1: 'Tue_ud', 2: 'Wed_ud', 3: 'Thu_ud', 4: 'Fri_ud'})
    price_list.append(this_price)
    updown_list.append(this_up_down)

In [118]:
cprice_df = pd.concat(price_list, axis=1).T.sort_index()
updown_df = pd.concat(updown_list, axis=1).T.sort_index()

In [182]:
cprice_df

,0,1,2,3,4
0050,87.15,88.05,88.00,87.25,87.45
0051,32.49,33.00,32.72,32.75,32.44
0052,57.75,58.50,59.80,59.00,57.90
0053,37.38,37.92,37.44,37.20,37.44
0054,24.63,24.94,24.05,24.63,24.70
0055,17.10,17.10,17.00,17.00,17.02
0056,26.00,26.75,26.17,26.32,26.45
0057,52.40,52.90,52.10,52.90,52.05
0058,46.56,47.02,47.27,47.27,47.37
0059,41.90,42.07,41.44,41.44,41.90


In [203]:
def dump_submit(cprice: pd.DataFrame, updown: pd.DataFrame) -> int:
    submit_df = pd.DataFrame()
    for row_index in range(len(cprice)):
        cprice_series = cprice.iloc[row_index]
        cprice_series = cprice_series.rename({0: 'Mon_cprice', 1: 'Tue_cprice', 2: 'Wed_cprice', 3: 'Thu_cprice', 4: 'Fri_cprice'})
        updown_series = updown.iloc[row_index]
        updown_series = updown_series.rename({0: 'Mon_ud', 1: 'Tue_ud', 2: 'Wed_ud', 3: 'Thu_ud', 4: 'Fri_ud'})
        
        cprice_name = cprice_series.axes[0]
        updown_name = updown_series.axes[0]
        this_series = pd.Series()
        for idx, a in enumerate(cprice_series):
            this_series = this_series.append(pd.Series(a, index=[cprice_name[idx]]))
            this_series = this_series.append(pd.Series(updown_series[updown_name[idx]], index=[updown_name[idx]]))
        this_series.name = cprice_series.name
        print(this_series)
        submit_df = pd.concat([submit_df, this_series], axis=1)
    return submit_df.T


qq= dump_submit(cprice_df, updown_df)

Mon_cprice    87.15
Mon_ud         1.00
Tue_cprice    88.05
Tue_ud         1.00
Wed_cprice    88.00
Wed_ud        -1.00
Thu_cprice    87.25
Thu_ud        -1.00
Fri_cprice    87.45
Fri_ud         1.00
Name: 0050, dtype: float64
Mon_cprice    32.49
Mon_ud         1.00
Tue_cprice    33.00
Tue_ud         1.00
Wed_cprice    32.72
Wed_ud         1.00
Thu_cprice    32.75
Thu_ud         1.00
Fri_cprice    32.44
Fri_ud        -1.00
Name: 0051, dtype: float64
Mon_cprice    57.75
Mon_ud        -1.00
Tue_cprice    58.50
Tue_ud         1.00
Wed_cprice    59.80
Wed_ud         1.00
Thu_cprice    59.00
Thu_ud        -1.00
Fri_cprice    57.90
Fri_ud        -1.00
Name: 0052, dtype: float64
Mon_cprice    37.38
Mon_ud         1.00
Tue_cprice    37.92
Tue_ud         1.00
Wed_cprice    37.44
Wed_ud        -1.00
Thu_cprice    37.20
Thu_ud        -1.00
Fri_cprice    37.44
Fri_ud         1.00
Name: 0053, dtype: float64
Mon_cprice    24.63
Mon_ud         1.00
Tue_cprice    24.94
Tue_ud         1.00
Wed_cprice  

In [204]:
qq

,Fri_cprice,Fri_ud,Mon_cprice,Mon_ud,Thu_cprice,Thu_ud,Tue_cprice,Tue_ud,Wed_cprice,Wed_ud
0050,87.45,1.0,87.15,1.0,87.25,-1.0,88.05,1.0,88.00,-1.0
0051,32.44,-1.0,32.49,1.0,32.75,1.0,33.00,1.0,32.72,1.0
0052,57.90,-1.0,57.75,-1.0,59.00,-1.0,58.50,1.0,59.80,1.0
0053,37.44,1.0,37.38,1.0,37.20,-1.0,37.92,1.0,37.44,-1.0
0054,24.70,1.0,24.63,1.0,24.63,1.0,24.94,1.0,24.05,-1.0
0055,17.02,1.0,17.10,-1.0,17.00,0.0,17.10,0.0,17.00,-1.0
0056,26.45,1.0,26.00,-1.0,26.32,1.0,26.75,1.0,26.17,-1.0
0057,52.05,-1.0,52.40,1.0,52.90,1.0,52.90,1.0,52.10,-1.0
0058,47.37,1.0,46.56,1.0,47.27,0.0,47.02,1.0,47.27,1.0
0059,41.90,1.0,41.90,-1.0,41.44,0.0,42.07,1.0,41.44,-1.0


In [171]:
cprice_name = cprice_series.axes[0]
updown_name = updown_series.axes[0]
this_series = pd.Series()
this_series.name = cprice_series.name

for idx, a in enumerate(cprice_series):
    this_series = this_series.append(pd.Series(a, index=[cprice_name[idx]]))
    this_series = this_series.append(pd.Series(updown_series[updown_name[idx]], index=[updown_name[idx]]))

In [172]:
this_series

Mon_cprice    31.22
Mon_ud        -1.00
Tue_cprice    31.10
Tue_ud        -1.00
Wed_cprice    31.07
Wed_ud        -1.00
Thu_cprice    31.14
Thu_ud         1.00
Fri_cprice    31.64
Fri_ud         1.00
dtype: float64

In [132]:
for a,b in zip(cprice_series, updown_series):
    cprice_name = cprice_series.axes[0]
    updown_series = updown_series.axes[0]
    
    this_series = pd.Series()
    this_series.name = cprice_series.name
    print(a)
    this_series = this_series.append(pd.Series(a, index=[101]))
    print(this_series)

31.22
Series([], Name: 00713, dtype: float64)
31.1
Series([], Name: 00713, dtype: float64)
31.07
Series([], Name: 00713, dtype: float64)
31.14
Series([], Name: 00713, dtype: float64)
31.64
Series([], Name: 00713, dtype: float64)


In [136]:
this_series.add(cprice_series)

Mon_cprice   NaN
Tue_cprice   NaN
Wed_cprice   NaN
Thu_cprice   NaN
Fri_cprice   NaN
Name: 00713, dtype: float64

In [144]:
?this_series.add

In [156]:
cprice_series['Mon_cprice']

31.219999999999999

In [162]:
cprice_series.axes[0]

Index(['Mon_cprice', 'Tue_cprice', 'Wed_cprice', 'Thu_cprice', 'Fri_cprice'], dtype='object')

In [150]:
this_series.append(pd.Series(200, index=[101]))

0       31.64
101    200.00
dtype: float64

In [163]:
cprice_series

Mon_cprice    31.22
Tue_cprice    31.10
Wed_cprice    31.07
Thu_cprice    31.14
Fri_cprice    31.64
Name: 00713, dtype: float64